### 필요 라이브러리 임포트

In [ ]:
import requests
from bs4 import BeautifulSoup

#### 월요웹툰 웹페이지 정보 받아보기

In [ ]:
url = 'http://comic.naver.com/webtoon/weekdayList.nhn?week=mon'
g = requests.get(url)
g.content

### 월요웹툰 페이지 파싱하기

In [ ]:
soup = BeautifulSoup(g.content, 'lxml')

#### 웹툰 목록 가져오기

In [ ]:
toon_list = soup.find('ul', attrs={'class': 'img_list'})
toon_list = toon_list.find_all('li')
len(toon_list)

#### 웹툰 링크 가져오기

In [ ]:
def get_toon_link(item):
    thumb = item.find('div', attrs={'class': 'thumb'})
    link = thumb.find('a')
    link = link['href']
    return link

toon_links = [get_toon_link(item) for item in toon_list]
toon_links

#### 웹툰 제목 가져오기

In [ ]:
def get_toon_title(item):
    return item.find('dl').find('dt').find('a').get_text()

toon_titles = [get_toon_title(item) for item in toon_list]
toon_titles

#### 각 요일별 웹툰 링크 가져오기

In [ ]:
import time

weekday_names = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
base_url = 'http://comic.naver.com/webtoon/weekdayList.nhn?week='

titles = []
links = []

for weekday in weekday_names:
    url = base_url + weekday
    g = requests.get(url)
    soup = BeautifulSoup(g.content, 'lxml')
    
    toon_list = soup.find('ul', attrs={'class': 'img_list'})
    toon_list = toon_list.find_all('li')
    
    toon_titles = [get_toon_title(item) for item in toon_list]
    titles.extend(toon_titles)
    
    toon_links = [get_toon_link(item) for item in toon_list]
    links.extend(toon_links)
    
    time.sleep(2)

#### 첫 번째 웹툰의 마지막 화 링크 알아보기

In [ ]:
link = links[0]
link

In [ ]:
toon_url = 'http://comic.naver.com' + link
toon_url

In [ ]:
g = requests.get(toon_url)
soup = BeautifulSoup(g.content, 'lxml')

In [ ]:
ep_list = soup.find('table', attrs={'class': 'viewList'})
ep_list = ep_list.find_all('tr')

In [ ]:
ep_list[0]

In [ ]:
ep_list[1]

In [ ]:
ep_list[2]

In [ ]:
ep_list[2].find('td').find('a')['href']

ep_list의 첫 번째 아이템은 에피소드 목록의 헤더이다. 

두 번째부터 에피소드 목록이 시작되는데, 실제 에피소드 목록이 시작되기 전에 광고가 삽입된 경우가 있다. 

광고를 나타내는 줄은 `<tr>` 태그의 attribute로 `class`를 가진다.

이 정보를 사용해 목록의 각 아이템이 광고인지 아닌지를 알아낼 수 있다.

In [ ]:
def get_last_ep_link(ep_list):
    for tr in ep_list[1:]:
        if 'class' not in tr.attrs.keys():
            return tr.find('td').find('a')['href']


last_ep_link = get_last_ep_link(ep_list)
last_ep_link

#### 전체 웹툰 마지막 화 받아오기

In [ ]:
last_eps = []

for i, link in enumerate(links):
    print(i+1, '/', len(links))
    
    toon_url = 'http://comic.naver.com' + link
    g = requests.get(toon_url)
    soup = BeautifulSoup(g.content, 'lxml')
    
    ep_list = soup.find('table', attrs={'class': 'viewList'})
    ep_list = ep_list.find_all('tr')
    
    last_ep = get_last_ep_link(ep_list)
    last_eps.append(last_ep)
    
    time.sleep(2)

#### 첫 번째 웹툰의 마지막 화 회별 별점 및 참여자 수 받아오기

In [ ]:
last_ep = last_eps[0]
last_ep = 'http://comic.naver.com' + last_ep

g = requests.get(last_ep)
soup = BeautifulSoup(g.content, 'lxml')

rating = soup.find('span', attrs={'id': 'topPointTotalNumber'})
rating = rating.get_text()

num_partic = soup.find('span', attrs={'class': 'pointTotalPerson'})
num_partic = num_partic.find('em').get_text()

#### 19금 웹툰의 경우

In [ ]:
last_ep = 'http://comic.naver.com/webtoon/detail.nhn?titleId=530312&no=142&weekday=mon'
g = requests.get(last_ep)
soup = BeautifulSoup(g.content, 'lxml')
print(soup.find('title'))

#### 전체 웹툰 마지막 화 회별 별점 및 참여자 수 받아오기

In [ ]:
ratings = []
num_partics = []

for i, last_ep in enumerate(last_eps):
    print(i + 1, '/', len(last_eps))
    
    last_ep = 'http://comic.naver.com' + last_ep
    g = requests.get(last_ep)
    soup = BeautifulSoup(g.content, 'lxml')
    
    title = soup.find('title').get_text()
    if '로그인' in title:
        ratings.append(-1)
        num_partics.append(-1)
        continue

    rating = soup.find('span', attrs={'id': 'topPointTotalNumber'})
    rating = rating.get_text()
    ratings.append(float(rating))

    num_partic = soup.find('span', attrs={'class': 'pointTotalPerson'})
    num_partic = num_partic.find('em').get_text()
    num_partics.append(int(num_partic))
    
    time.sleep(2)

#### 요일 분류하기

In [ ]:
weekdays = [link[-3:] for link in links]

#### pandas DataFrame으로 변환하기

In [ ]:
data_dict = {
    'weekday': weekdays,
    'title': titles,
    'rating': ratings,
    'num_partics': num_partics
}

import pandas as pd
df = pd.DataFrame(data_dict)

#### 미수집 웹툰 필터링

In [ ]:
df = df[df['num_partics'] > 0]

#### Top 20 참여자 웹툰 확인하기

In [ ]:
df_top_20 = df.sort_values(by='num_partics', ascending=False)[:20]
df_top_20

#### Top 20 참여자 웹툰 시각화

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('font', family='AppleGothic')
matplotlib.style.use('ggplot')

In [ ]:
df_top_20.plot.barh(x='title', 
                    y='num_partics', 
                    figsize=(15, 10))

#### 요일별 별점 참여자 평균/표준편차 시각화

In [ ]:
day_mean = df.groupby('weekday')['num_partics'].mean()
day_std = df.groupby('weekday')['num_partics'].std()

In [ ]:
df2 = pd.DataFrame(index=weekday_names)
df2['mean'] = day_mean
df2['std'] = day_std

In [ ]:
df2.plot.bar(figsize=(10, 8))